In [10]:
from google.colab import files, drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
#Import needed packages
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import numpy as np


Define the network structure

In [12]:


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class Net(nn.Module):
    def __init__(self, num_classes=7):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()

        self.pool = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()

        self.conv4 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()

        self.fc = nn.Linear(in_features=16 * 16 * 24, out_features=num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.relu1(output)

        output = self.conv2(output)
        output = self.relu2(output)

        output = self.pool(output)

        output = self.conv3(output)
        output = self.relu3(output)

        output = self.conv4(output)
        output = self.relu4(output)

        output = output.view(-1, 16 * 16 * 24)

        output = self.fc(output)

        return output

Import the raw images, with necessary resize and convert into standarzied RGB *format*

In [13]:

#create customized dataset
import os
import glob
import numpy as np
from skimage import io
import re
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import re

class Lung_Pnemonia(Dataset):

  def __init__(self,dir,transform=None):
    self.dir=dir
    self.transform=transform

  def __len__(self):
    files=glob.glob(self.dir+'/*.jpeg')[:1000]
    #files.extend(glob(self.dir+'/*.png'))
    #files.extend(glob(self.dir+'/*.jpg'))
    return len(files)



  def __getitem__(self,idx):
    if torch.is_tensor(idx):
      idx=idx.tolist()

    all_instances=glob.glob(self.dir+'/*.jpeg')[:1000] #return a list of file names
    read = lambda imname: np.asarray(Image.open(imname).resize((32,32), resample = Image.BILINEAR).convert("RGB")) 
    ims = read(os.path.join(self.dir, all_instances[idx])) 
    im_array=np.array(ims)   
    #print(all_instances[idx])
    if bool(re.search(('IM'), all_instances[idx])):
      label=np.array(0)
    elif bool(re.search(('virus'), all_instances[idx])):
      label=np.array(1)
    elif bool(re.search(('bacteria'), all_instances[idx])):
      label=np.array(2)
    elif bool(re.search(('streptococcus'), all_instances[idx])):
      label=np.array(3)
    elif bool(re.search(('acute'), all_instances[idx])):
      label=np.array(4)
    elif bool(re.search(('SARS'), all_instances[idx])):
      label=np.array(5)
    else:
      label=np.array(6)

    instance={'image':im_array, 'label':label}      
    if self.transform:
      instance['image']= self.transform(instance['image'])

    return instance
    

create train/test data loader

In [14]:
#create train/val data loader
from torch.utils.data import random_split
from torchvision import transforms, utils

batch_size=28

trainset=Lung_Pnemonia('/content/drive/My Drive/Coronahack-Chest-XRay-Dataset/train',
                       transform=transforms.Compose([transforms.ToTensor(),
                                                     transforms.Normalize((0.5,), (0.5,))]))

train_loader=DataLoader(trainset,batch_size,shuffle=True,num_workers=1)

testset=Lung_Pnemonia('/content/drive/My Drive/Coronahack-Chest-XRay-Dataset/test',transform=transforms.Compose([transforms.ToTensor(),
                                                                                                                 transforms.Normalize((0.5,), (0.5,))]))

test_loader=DataLoader(testset,batch_size,shuffle=True,num_workers=1)

define optimizer

In [15]:

from torch.optim import Adam


# Check if gpu support is available
cuda_avail = torch.cuda.is_available()

# Create model, optimizer and loss function
model = Net(num_classes=7)

#if cuda is available, move the model to the GPU
if cuda_avail:
    model.cuda()

#Define the optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [16]:
device

device(type='cuda', index=0)

Define Train-Validation Process

In [17]:
def train_test(num_epochs):
  best_acc = 0.0

  for epoch in range(num_epochs):
    model.train()
    train_acc = 0.0
    train_loss = 0.0
    for i, batch in enumerate(train_loader):
      if cuda_avail:
        #images = Variable(images.cuda())
        #labels = Variable(labels.cuda())
        inputs, targets = batch['image'].to(device,dtype=torch.float), batch['label'].to(device,dtype=torch.long)
        #print(type(inputs))
            # Clear all accumulated gradients
      optimizer.zero_grad()
            # Predict classes using images from the test set
      outputs = model(inputs)
            # Compute the loss based on the predictions and actual labels
      loss = loss_fn(outputs, targets)
            # Backpropagate the loss
      loss.backward()

            # Adjust parameters according to the computed gradients
      optimizer.step()

      train_loss += loss.cuda().data * inputs.size(0)
      _, prediction = torch.max(outputs.data, 1)
            
      train_acc += torch.sum(prediction == targets.data)



        # Evaluate on the test set
    model.eval()

    test_acc = 0.0
    test_loss = 0.0
    with torch.no_grad():
      for b, batch in enumerate(test_loader):
        inputs, targets = batch['image'].to(device,dtype=torch.float), batch['label'].to(device,dtype=torch.long)
        outputs = model(inputs)
        #loss = loss_fn(outputs, targets)
        
        # update-average-validation-loss 
        #test_loss += loss.cuda().data * inputs.size(0)
        _, prediction = torch.max(outputs.data, 1)
            
        test_acc += torch.sum(prediction == targets.data)

    mean_train_acc = train_acc/len(train_loader.sampler)   
    mean_train_loss = train_loss/len(train_loader.sampler)
    mean_test_acc = test_acc/len(test_loader.sampler)   


        # Print the metrics
    print("Epoch {}, Train Accuracy: {} , TrainLoss: {} , Test Accuracy: {}".format(epoch+1, mean_train_acc, mean_train_loss,mean_test_acc))

                        

Train 20 epochs (or more if enough computation resources allocated)

In [18]:
if __name__ == "__main__":
  train_test(20)


Epoch 1, Train Accuracy: 0.5360000133514404 , TrainLoss: 0.8234519362449646 , Test Accuracy: 0.3878205120563507
Epoch 2, Train Accuracy: 0.6390000581741333 , TrainLoss: 0.6575329899787903 , Test Accuracy: 0.44871795177459717
Epoch 3, Train Accuracy: 0.6810000538825989 , TrainLoss: 0.6136254072189331 , Test Accuracy: 0.5240384936332703
Epoch 4, Train Accuracy: 0.7410000562667847 , TrainLoss: 0.5592641830444336 , Test Accuracy: 0.5
Epoch 5, Train Accuracy: 0.7550000548362732 , TrainLoss: 0.5337328314781189 , Test Accuracy: 0.5416666865348816
Epoch 6, Train Accuracy: 0.76500004529953 , TrainLoss: 0.5125279426574707 , Test Accuracy: 0.5480769276618958
Epoch 7, Train Accuracy: 0.7850000262260437 , TrainLoss: 0.46403923630714417 , Test Accuracy: 0.5288461446762085
Epoch 8, Train Accuracy: 0.7990000247955322 , TrainLoss: 0.4430633783340454 , Test Accuracy: 0.5288461446762085
Epoch 9, Train Accuracy: 0.8020000457763672 , TrainLoss: 0.4357111155986786 , Test Accuracy: 0.5336538553237915
Epoch 1